In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 51.9 MB/s eta 0:00:00


In [ ]:
!pip install polars

In [ ]:
import random
from collections import defaultdict, Counter
from typing import List, Dict, Union
import pickle

import numpy as np
import implicit
from scipy.sparse import lil_matrix
from tqdm import tqdm
import pandas as pd
import polars as pl

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [ ]:
TOP_N = 50
LOCALE = "UK"
VER = "01"
DIR = "/content/drive/MyDrive/kddcup2023/"

# parameters for IMF
FACTORS = 50
LAMBDA = 0.01
ITERATIONS = 500
SEED = 42

In [ ]:
def preprocess(df:pl.DataFrame) -> pl.DataFrame:
    df = df.explode(["prev_items"])
    df = df.with_columns(
        df.select(pl.col("session_id").cumcount().over("session_id").alias("sequence_num"))
    )
    return df

In [ ]:
def train_imf_and_generate_candidates(df:pd.DataFrame):
    # generate sparse matrix
    unique_user_ids = sorted(df["session_id"].unique())
    unique_item_ids = sorted(df["prev_items"].unique())
    user_id2index = dict(zip(unique_user_ids, range(len(unique_user_ids))))
    item_id2index = dict(zip(unique_item_ids, range(len(unique_item_ids))))

    matrix = lil_matrix((len(unique_user_ids), len(unique_item_ids)))
    for _, row in tqdm(df.iterrows()):
        user_index = user_id2index[row["session_id"]]
        item_index = item_id2index[row["prev_items"]]
        matrix[user_index, item_index] = 1.0

    # convert LIL to CSR
    matrix = matrix.tocsr()

    # model initialization
    model = implicit.bpr.BayesianPersonalizedRanking(
        factors = FACTORS,
        regularization = LAMBDA,
        iterations = ITERATIONS,
        random_state = SEED,
    )

    # train model
    model.fit(matrix)

    # generate candidate
    users = []
    candidates = []
    imf_scores = []

    for user_id, user_index in tqdm(user_id2index.items()):
        item_indexes, scores = model.recommend(user_index, matrix[user_index], N=TOP_N, filter_already_liked_items=True)
        for item_index, score in zip(item_indexes, scores):
            users.append(user_id)
            candidates.append(unique_item_ids[item_index])
            imf_scores.append(score)

    candidates = pd.DataFrame({
        "session_id": users,
        "candidate_item": candidates,
        "bpr_score": imf_scores,
    })

    candidates = pl.from_pandas(candidates)
    candidates = candidates \
    .sort(["session_id", "bpr_score"], descending=[False, True]) \
    .with_columns(pl.col("bpr_score").rank(descending=True, method="min").over("session_id").alias("bpr_rank"))
    candidates = candidates.to_pandas()

    return model, user_id2index, item_id2index, candidates

## For local train/eval

In [ ]:
train = pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/preprocessed/task1/train_task1.parquet")
test = pl.read_parquet("/content/drive/MyDrive/kddcup2023/data/preprocessed/task1/test_task1_phase2.parquet")

train = preprocess(train)
test = preprocess(test)
session_df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])
session_df = session_df.to_pandas()
session_df = session_df[session_df["locale"] == LOCALE]

In [ ]:
%%time
model, user_id2index, item_id2index, candidates = train_imf_and_generate_candidates(session_df)

5336525it [04:53, 18153.39it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 1298118/1298118 [2:20:56<00:00, 153.50it/s]


CPU times: user 11h 37min 18s, sys: 6h 42min 19s, total: 18h 19min 37s
Wall time: 2h 31min 30s


In [ ]:
# save models
model_name = f"bpr_{VER}_{LOCALE}_model_for_train_or_eval.npz"
user_ids_name = f"bpr_{VER}_{LOCALE}_user_id2index_for_train_or_eval.pickle"
item_ids_name = f"bpr_{VER}_{LOCALE}_item_id2index_for_train_or_eval.pickle"
model.save(DIR + "models/task1/" + model_name)
with open(DIR + "models/task1/" + user_ids_name, "wb") as f:
    pickle.dump(user_id2index, f)
with open(DIR + "models/task1/" + item_ids_name, "wb") as f:
    pickle.dump(item_id2index, f)

# save candidates
file_name = f"bpr_{VER}_{LOCALE}_for_train_or_eval.parquet"
candidates.to_parquet(DIR + "data/interim/candidates/task1/" + file_name)

In [ ]:
candidates.head()

# For test inference

In [ ]:
train = pl.read_parquet("/content/drive/MyDrive/kddcup2023-master/data/preprocessed/task1/train_task1.parquet")
test = pl.read_parquet("/content/drive/MyDrive/kddcup2023-master/data/preprocessed/task1/test_task1_phase2.parquet")

In [ ]:
# Append train's next_item to prev_items
prev_items_list = train["prev_items"].to_list()
next_item_list = train["next_item"].to_list()
prev_items_list_updated = []
for prev_items, next_item in zip(prev_items_list, next_item_list):
    prev_items.append(next_item)
    prev_items_list_updated.append(prev_items)

train = train.with_columns(
    pl.Series(name="prev_items", values=prev_items_list_updated)
)

In [ ]:
train = preprocess(train)
test = preprocess(test)
session_df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])
session_df = session_df.to_pandas()
session_df = session_df[session_df["locale"] == LOCALE]

In [ ]:
%%time
model, user_id2index, item_id2index, candidates = train_imf_and_generate_candidates(session_df)

6518706it [06:27, 16812.51it/s]


  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 1298118/1298118 [2:54:13<00:00, 124.19it/s]


CPU times: user 14h 19min 32s, sys: 8h 6min 11s, total: 22h 25min 44s
Wall time: 3h 8min


In [ ]:
# save models
model_name = f"bpr_{VER}_{LOCALE}_model_for_inference.npz"
user_ids_name = f"bpr_{VER}_{LOCALE}_user_id2index_for_inference.pickle"
item_ids_name = f"bpr_{VER}_{LOCALE}_item_id2index_for_inference.pickle"
model.save(DIR + "models/task1/" + model_name)
with open(DIR + "models/task1/" + user_ids_name, "wb") as f:
    pickle.dump(user_id2index, f)
with open(DIR + "models/task1/" + item_ids_name, "wb") as f:
    pickle.dump(item_id2index, f)

# save candidates
file_name = f"bpr_{VER}_{LOCALE}_for_inference.parquet"
candidates.to_parquet(DIR + "data/interim/candidates/task1/" + file_name)

In [ ]:
candidates.head()

,session_id,candidate_item,bpr_score,bpr_rank
0,test_phase2_201035,B08KXPF5NK,3.581294,1
1,test_phase2_201035,B07B8Y222P,3.558642,2
2,test_phase2_201035,B07KW2P3B8,3.526233,3
3,test_phase2_201035,B09KLZ1R81,3.520002,4
4,test_phase2_201035,B09KXT2BBS,3.517550,5
